# Input Output Data Handling

First you need to understand the difference between "State Input" and "Action Input". The "State Input" is just the "State Output" from previous state. And the Action is the underlying computation, such as Lambda, ECS, Glue. The "Action Input" is the raw JSON input for the computation.

AWS Step Function Provides the following method to manipulate the Input/Output data:

- ``InputPath``: convert "State Input" to "Action Input", select single JSON node.
- ``Parameters``: convert "State Input" to "Action Input", use payload template language to construct the "Action Input".
- ``ResultSelector``: convert "Action Output" to "State Output", use payload template language to construct the "State Output".
- ``ResultPath``: convert "Action Output" to "State Output", select single JSON node.

![input-output-processing](https://docs.aws.amazon.com/step-functions/latest/dg/images/input-output-processing.png)

In this example, we create a simple EShop purchase use case. First we would like to convert the state input from the API, extract the payment information, and "Verify Payment". If payment is verified, we would like to forward the "order_id" information and send to "Process Order". If payment is NOT verified, we fail the workflow.

In [5]:
import aws_stepfunction as sfn
from rich import print

![input-output-sample-workflow](./input-output.png)

In [6]:
workflow = sfn.Workflow()

verify_payment = sfn.Task(
    id="Verify Payment",
    resource="arn:...",
    parameters={
        "credit_card_number": "$.payment.card_number",
        "order_id": "$.order.order_id",
    },
    result_selector={
        "verified": "$.verify_result.result",
        "order_id": "$.order_id",
    },
)

process_order = sfn.Task(
    id="Process Order",
    resource="arn:...",
    parameters={
        "order_id": "$.order_id",
    },
)

In [7]:
(
    workflow.start_from(verify_payment)
    .choice(
        [
            sfn.Var("$.verified").boolean_equals(True)
            .next_then(process_order)
        ],
        default=sfn.Fail(),
    )
)

workflow.continue_from(process_order).end()

print(workflow.serialize())

{
    'StartAt': 'Verify Payment',
    'States': {
        'Verify Payment': {
            'Type': 'Task',
            'Resource': 'arn:...',
            'Next': 'Choice-by-Verify Payment',
            'Parameters': {'credit_card_number': '$.payment.card_number', 'order_id': '$.order.order_id'},
            'ResultSelector': {'verified': '$.verify_result.result', 'order_id': '$.order_id'}
        },
        'Choice-by-Verify Payment': {
            'Type': 'Choice',
            'Choices': [{'Variable': '$.verified', 'BooleanEquals': True, 'Next': 'Process Order'}],
            'Default': 'Fail-0c3c4cb'
        },
        'Process Order': {
            'Type': 'Task',
            'Resource': 'arn:...',
            'End': True,
            'Parameters': {'order_id': '$.order_id'}
        },
        'Fail-0c3c4cb': {'Type': 'Fail'}
    }
}